In [16]:
import random
import numpy as np
import json
from util import *

In [17]:
def assign_riders_with_weighted_probability(riders, effectiveness_indicator):
    # 가중치에 따라 라이더 타입을 선택
    total_effectiveness = sum([indicator[1] for indicator in effectiveness_indicator])
    weights = [indicator[1] / total_effectiveness for indicator in effectiveness_indicator]

    # 라이더 타입 리스트와 그에 해당하는 가중치 리스트를 생성
    rider_types = [indicator[0] for indicator in effectiveness_indicator]
    #print(rider_types)

    while rider_types:
        selected_type = random.choices(rider_types, weights=weights, k=1)[0]
        print(f"selected_type : {selected_type}")
        
        # 해당 타입의 가용 가능한 라이더 찾기
        selected_rider = next((rider for rider in riders if rider.type == selected_type), None)
        print(f"selected_rider : {selected_rider}")
        
        if selected_rider:
            return selected_rider
        else:
            # 선택된 타입의 라이더가 가용 가능하지 않다면, 해당 타입을 리스트에서 제거
            index = rider_types.index(selected_type)
            rider_types.pop(index)
            weights.pop(index)
            # 가중치를 다시 계산 (남은 가중치의 합이 1이 되도록)
            if weights:
                total_weights = sum(weights)
                weights = [w / total_weights for w in weights]

    # 만약 모든 타입의 라이더가 가용하지 않다면 None 반환
    return None

In [18]:
problem_file = 'STAGE1_17.json'
# problem_file = "TEST_K50_1.json"
timelimit = 60

# np.random.seed(1)

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time)

In [19]:
def calculate_efficiencies(K, all_riders, all_orders, dist_mat):
    # 모든 주문의 부피 평균 계산
    order_volumes = [order.volume for order in all_orders]
    avg_volume = np.mean(order_volumes)
    print(avg_volume)

    # d1, d2, d3 계산
    d1 = np.sum(dist_mat[:K, :K]) / (K*K)
    d2 = np.sum(dist_mat[:K, K:2*K]) / (K*K)
    d3 = np.sum(dist_mat[K:2*K, K:2*K]) / (K*K)

    # 각 배달원의 효율성 지표 계산 함수
    def calculate_efficiency(rider, avg_volume, d1, d2, d3):
        capacity = rider.capa
        variable_cost = rider.var_cost
        fixed_cost = rider.fixed_cost
        
        Ri = (0.8 * capacity) / avg_volume
        print(f"Ri : {Ri}")
        Xi = (Ri - 1) * d1 + (Ri - 1) * d3 + d2
        print(f"Xi : {Xi}")
        efficiency = fixed_cost + (Xi / 100) * variable_cost
        
        return efficiency

    # 각 배달원의 효율성 지표 계산
    efficiencies = []
    for rider in all_riders:
        rider_type = rider.type
        efficiency = calculate_efficiency(rider, avg_volume, d1, d2, d3)
        efficiencies.append([rider_type, efficiency])

    # 효율성 지표 반환
    return efficiencies

calculate_efficiencies(K, ALL_RIDERS, ALL_ORDERS, DIST)

24.983333333333334
Ri : 3.202134756504336
Xi : 59689.63235166407
Ri : 2.2414943295530354
Xi : 38474.31910283152
Ri : 6.404269513008672
Xi : 130407.34318110593


[['BIKE', 40813.77941099844],
 ['WALK', 16542.29573084946],
 ['CAR', 135407.3431811059]]

In [20]:
def assign_riders_with_weighted_probability(riders, effectiveness_indicator):
    total_effectiveness = sum([indicator[1] for indicator in effectiveness_indicator])
    weights = [total_effectiveness / indicator[1] for indicator in effectiveness_indicator]
    selected_rider = random.choices(riders, weights=weights, k=1)[0]
    return selected_rider

effectiveness_indicator = calculate_efficiencies(K, ALL_RIDERS, ALL_ORDERS, DIST)
print(assign_riders_with_weighted_probability(ALL_RIDERS, effectiveness_indicator))

24.983333333333334
Ri : 3.202134756504336
Xi : 59689.63235166407
Ri : 2.2414943295530354
Xi : 38474.31910283152
Ri : 6.404269513008672
Xi : 130407.34318110593
Rider([WALK, 1.3227513227513228, 70, 30, 5000, 120, 90])


In [21]:
effectiveness_indicator = [["bike", 100], ["car", 500], ["walk", 300]]

In [22]:
assign_riders_with_weighted_probability(ALL_RIDERS, effectiveness_indicator)

Rider([BIKE, 5.291005291005291, 100, 60, 5000, 120, 60])

In [23]:
for rider in ALL_RIDERS:
    print(f"Rider Type: {rider.type}, Available Number: {rider.available_number}")


Rider Type: BIKE, Available Number: 60
Rider Type: WALK, Available Number: 90
Rider Type: CAR, Available Number: 300


In [24]:

def calculate_efficiencies(K, all_riders, all_orders, dist_mat):
    # 모든 주문의 부피 평균 계산
    order_volumes = [order.volume for order in all_orders]
    avg_volume = np.mean(order_volumes)

    # d1, d2, d3 계산
    d1 = np.sum(dist_mat[:K, :K]) / 2500
    d2 = np.sum(dist_mat[:K, K:2*K]) / 2500
    d3 = np.sum(dist_mat[K:2*K, K:2*K]) / 2500

    # 각 배달원의 효율성 지표 계산 함수
    def calculate_efficiency(rider, avg_volume, d1, d2, d3):
        capacity = rider.capa
        variable_cost = rider.var_cost
        fixed_cost = rider.fixed_cost
        
        Ri = (0.8 * capacity) / avg_volume
        Xi = (Ri - 1) * d1 + (Ri - 1) * d3 + d2
        efficiency = fixed_cost + (Xi / 100) * variable_cost
        
        return efficiency

    # 각 배달원의 효율성 지표 계산
    efficiencies = []
    for rider in all_riders:
        rider_type = rider.type
        efficiency = calculate_efficiency(rider, avg_volume, d1, d2, d3)
        efficiencies.append([rider_type, efficiency])

    # 효율성 지표 반환
    return efficiencies

In [25]:
calculate_efficiencies(K, ALL_RIDERS, ALL_ORDERS, DIST)

[['BIKE', 1294296.0587959439],
 ['WALK', 420522.6463105804],
 ['CAR', 4699664.354519813]]

In [26]:
import json
import numpy as np
import csv

# 부피와 마감 시간의 평균을 계산하는 함수
def calculate_avg_volume_and_deadline(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    # 주문 리스트에서 7번째 인덱스 (부피)와 8번째 인덱스 (마감 시간)를 추출
    volumes = [order[7] for order in data['ORDERS']]
    deadlines = [order[8] for order in data['ORDERS']]
    
    # 부피와 마감 시간의 평균 계산
    avg_volume = np.mean(volumes)
    avg_deadline = np.mean(deadlines)
    
    return avg_volume, avg_deadline

# 결과를 저장할 리스트
results = []

# 1부터 18까지의 파일에 대해 평균 계산
for i in range(1, 19):
    file_path = f'STAGE1_{i}.json'
    avg_volume, avg_deadline = calculate_avg_volume_and_deadline(file_path)
    results.append([file_path, avg_volume, avg_deadline])
    print(f"{file_path}의 평균 부피: {avg_volume}, 평균 마감 시간: {avg_deadline}")

# CSV 파일로 저장
csv_file = 'average_volumes_and_deadlines.csv'
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["File", "Average Volume", "Average Deadline"])
    writer.writerows(results)

print(f"Results have been saved to {csv_file}")



STAGE1_1.json의 평균 부피: 24.33, 평균 마감 시간: 4228.53
STAGE1_2.json의 평균 부피: 26.36, 평균 마감 시간: 5318.1
STAGE1_3.json의 평균 부피: 25.155, 평균 마감 시간: 4859.655
STAGE1_4.json의 평균 부피: 27.83, 평균 마감 시간: 4722.39
STAGE1_5.json의 평균 부피: 23.21, 평균 마감 시간: 4130.883333333333
STAGE1_6.json의 평균 부피: 25.483333333333334, 평균 마감 시간: 5060.046666666667
STAGE1_7.json의 평균 부피: 27.52, 평균 마감 시간: 4499.99
STAGE1_8.json의 평균 부피: 26.78, 평균 마감 시간: 4464.47
STAGE1_9.json의 평균 부피: 31.765, 평균 마감 시간: 4944.17
STAGE1_10.json의 평균 부피: 27.715, 평균 마감 시간: 4690.775
STAGE1_11.json의 평균 부피: 29.553333333333335, 평균 마감 시간: 4922.606666666667
STAGE1_12.json의 평균 부피: 30.433333333333334, 평균 마감 시간: 5567.913333333333
STAGE1_13.json의 평균 부피: 27.85, 평균 마감 시간: 4419.4
STAGE1_14.json의 평균 부피: 29.18, 평균 마감 시간: 5992.57
STAGE1_15.json의 평균 부피: 29.035, 평균 마감 시간: 5074.285
STAGE1_16.json의 평균 부피: 25.17, 평균 마감 시간: 5896.625
STAGE1_17.json의 평균 부피: 24.983333333333334, 평균 마감 시간: 4661.99
STAGE1_18.json의 평균 부피: 24.81, 평균 마감 시간: 5840.823333333334
Results have been saved to average_vo

In [27]:
import numpy as np
import time
import itertools
import random
from itertools import permutations
from util import Bundle, try_merging_bundles, get_total_distance, get_total_volume, test_route_feasibility, get_cheaper_available_riders, try_bundle_rider_changing
import concurrent.futures

In [28]:
def calculate_efficiencies(K, all_riders, all_orders, dist_mat):
    # 모든 주문의 부피 평균 계산
    order_volumes = [order.volume for order in all_orders]
    avg_volume = np.mean(order_volumes)

    # d1, d2, d3 계산
    d1 = np.sum(dist_mat[:K, :K]) / 2500
    d2 = np.sum(dist_mat[:K, K:2*K]) / 2500
    d3 = np.sum(dist_mat[K:2*K, K:2*K]) / 2500

    # 각 배달원의 효율성 지표 계산 함수
    def calculate_efficiency(rider, avg_volume, d1, d2, d3):
        capacity = rider.capa
        variable_cost = rider.var_cost
        fixed_cost = rider.fixed_cost
        
        Ri = (0.8 * capacity) / avg_volume
        Xi = (Ri - 1) * d1 + (Ri - 1) * d3 + d2
        efficiency = fixed_cost + (Xi / 100) * variable_cost
        
        return efficiency

    # 각 배달원의 효율성 지표 계산
    efficiencies = []
    for rider in all_riders:
        rider_type = rider.type
        efficiency = calculate_efficiency(rider, avg_volume, d1, d2, d3)
        efficiencies.append([rider_type, efficiency])

    # 효율성 지표 반환
    return efficiencies

In [29]:
def find_nearest_orders(current_bundle, remaining_orders, dist_mat, K, num_orders=30):
    bundle_size = len(current_bundle)
    
    distances = []
    for order in remaining_orders:
        total_distance = 0
        for existing_order in current_bundle:
            pickup_distance = dist_mat[existing_order.id, order.id]
            delivery_distance = dist_mat[existing_order.id + K, order.id + K]
            total_distance += (pickup_distance + delivery_distance)
        average_distance = total_distance / bundle_size
        distances.append((order, average_distance))
    
    distances.sort(key=lambda x: x[1])
    nearest_orders = [order for order, _ in distances[:num_orders]]
    return nearest_orders

In [30]:
def assign_riders_with_weighted_probability(riders, effectiveness_indicator):
    total_effectiveness = sum([indicator[1] for indicator in effectiveness_indicator])
    weights = [total_effectiveness / indicator[1] for indicator in effectiveness_indicator]
    selected_rider = random.choices(riders, weights=weights, k=1)[0]
    return selected_rider

In [31]:
def assign_orders_to_rider(rider, orders, dist_mat, K, all_orders):
    bundles = []
    remaining_orders = orders[:]
    
    while remaining_orders and rider.available_number > 0:
        current_order = remaining_orders.pop(0)
        current_bundle = [current_order]
        shop_seq = [current_order.id]
        delivery_seq = sorted(shop_seq, key=lambda order_id: all_orders[order_id].deadline)
        
        is_feasible = test_route_feasibility(all_orders, rider, shop_seq, delivery_seq)
        if is_feasible != 0:
            remaining_orders.insert(0, current_order)
            return bundles, remaining_orders
        
        current_volume = current_order.volume
        current_time = current_order.ready_time

        while True:
            nearest_orders = find_nearest_orders(current_bundle, remaining_orders, dist_mat, K, 30)
            added = False

            if len(current_bundle) >= 4:
                break
            
            for next_order in nearest_orders:
                if current_volume + next_order.volume > rider.capa:
                    continue
                
                current_bundle_ids = [o.id for o in current_bundle]
                next_bundle_ids = [next_order.id]

                combined_ids = current_bundle_ids + next_bundle_ids
                pickup_permutations = itertools.permutations(combined_ids)

                valid_combinations = []
                
                for perm_shop_seq in pickup_permutations:
                    delivery_permutations = itertools.permutations(perm_shop_seq)
                    for perm_dlv_seq in delivery_permutations:
                        new_bundle = Bundle(all_orders, rider, list(perm_shop_seq), list(perm_dlv_seq), current_volume + next_order.volume, 0)
                        new_bundle.total_dist = get_total_distance(K, dist_mat, list(perm_shop_seq), list(perm_dlv_seq))
                        new_bundle.update_cost()

                        is_feasible = test_route_feasibility(all_orders, rider, list(perm_shop_seq), list(perm_dlv_seq))
                        if is_feasible == 0:
                            valid_combinations.append((list(perm_shop_seq), list(perm_dlv_seq)))

                if valid_combinations:
                    best_combination = min(valid_combinations, key=lambda x: get_total_distance(K, dist_mat, x[0], x[1]))
                    best_shop_seq, best_dlv_seq = best_combination

                    current_bundle.append(next_order)
                    current_volume += next_order.volume
                    current_time += rider.T[current_bundle[-2].id, next_order.id]
                    remaining_orders.remove(next_order)
                    added = True

                    # 선택된 best_shop_seq와 best_dlv_seq로 번들 갱신
                    shop_seq = best_shop_seq
                    delivery_seq = best_dlv_seq
                    break

            if not added:
                break

        final_bundle = Bundle(all_orders, rider, shop_seq, delivery_seq, current_volume, get_total_distance(K, dist_mat, shop_seq, delivery_seq))
        bundles.append(final_bundle)
        rider.available_number -= 1

    return bundles, remaining_orders

In [32]:
def single_run_algorithm(K, all_orders, all_riders, dist_mat, timelimit=60):
    start_time = time.time()

    for r in all_riders:
        r.T = np.round(dist_mat / r.speed + r.service_time)

    # 효율성 지표 계산
    effectiveness_indicator = calculate_efficiencies(K, all_riders, all_orders, dist_mat)

    # 주문들을 무작위로 섞어 정렬
    sorted_orders = random.sample(all_orders, len(all_orders))

    # 모든 라이더를 합쳐서 처리
    all_riders_list = all_riders

    all_bundles = []

    while sorted_orders and all_riders_list:
        rider = assign_riders_with_weighted_probability(all_riders_list, effectiveness_indicator)
        if rider.available_number > 0:
            bundles, sorted_orders = assign_orders_to_rider(rider, sorted_orders, dist_mat, K, all_orders)
            for bundle in bundles:
                all_bundles.append(bundle)

    # 최적의 비용 계산
    best_obj = sum((bundle.cost for bundle in all_bundles)) / K
    print(f'Initial best obj = {best_obj}')
    print(f"bundles: {bundles}")

    # # 번들에 하나의 주문만 있는 경우, 병합 시도
    # for i, bundle in enumerate(all_bundles):
    #     if len(bundle[1]) == 1:
    #         for j, other_bundle in enumerate(all_bundles):
    #             if i != j:
    #                 merged_bundle = try_merging_bundles(bundle, other_bundle, all_orders, dist_mat, K)
    #                 if merged_bundle and test_route_feasibility(all_orders, bundle.rider, merged_bundle.shop_seq, merged_bundle.dlv_seq) == 0:
    #                     # 병합이 가능하고 유효하면 번들을 병합
    #                     all_bundles[i] = merged_bundle
    #                     del all_bundles[j]
    #                     break

    # 병합 후 최종 비용 재계산
    best_obj = sum((bundle.cost for bundle in all_bundles)) / K
    print(f'Optimized obj after merging single-order bundles = {best_obj}')

    solution = [
        [bundle.rider.type, bundle.shop_seq, bundle.dlv_seq]
        for bundle in all_bundles
    ]
    print(solution)

    return solution, best_obj

In [33]:
import random
import numpy as np
import json
from util import *

In [34]:
problem_file = 'STAGE1_17.json'
# problem_file = "TEST_K50_1.json"
timelimit = 60

# np.random.seed(1)

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time)

In [35]:
best_solution = [['BIKE', [14, 125, 105], [14, 125, 105]], ['BIKE', [265, 293], [293, 265]], ['BIKE', [274, 279, 219], [219, 274, 279]], ['BIKE', [192, 175, 201], [175, 192, 201]], ['BIKE', [268, 187, 202], [202, 187, 268]], ['BIKE', [143, 223, 150], [150, 143, 223]], ['BIKE', [123, 242, 147, 206], [147, 242, 206, 123]], ['BIKE', [134, 246], [134, 246]], ['BIKE', [161, 135, 239, 198], [135, 239, 198, 161]], ['BIKE', [131, 250, 248], [248, 131, 250]], ['BIKE', [157, 158, 170], [158, 170, 157]], ['BIKE', [153, 124, 225, 109], [124, 153, 225, 109]], ['BIKE', [133, 220, 217], [217, 220, 133]], ['BIKE', [122, 169, 263], [263, 169, 122]], ['BIKE', [20, 52], [52, 20]], ['BIKE', [29, 32, 45], [32, 45, 29]], ['BIKE', [64, 43, 12], [43, 12, 64]], ['BIKE', [5, 10, 57, 87], [10, 5, 57, 87]], ['BIKE', [116, 205, 104], [116, 205, 104]], ['BIKE', [59, 174], [59, 174]], ['BIKE', [216, 183, 280, 277], [183, 216, 277, 280]], ['BIKE', [36, 4, 114], [4, 114, 36]], ['BIKE', [285, 269, 295], [295, 285, 269]], ['BIKE', [53, 68, 164], [68, 53, 164]], ['BIKE', [200, 228, 199, 128], [199, 200, 128, 228]], ['BIKE', [103, 67, 93], [103, 67, 93]], ['BIKE', [37, 17, 111, 92], [92, 37, 17, 111]], ['BIKE', [60, 119], [60, 119]], ['BIKE', [252, 283, 261], [252, 261, 283]], ['BIKE', [2, 1, 110], [1, 2, 110]], ['BIKE', [229, 138, 231], [138, 229, 231]], ['BIKE', [222, 151, 251], [222, 151, 251]], ['BIKE', [39, 118, 54, 132], [39, 54, 118, 132]], ['BIKE', [73, 16, 126], [16, 73, 126]], ['BIKE', [255, 288], [255, 288]], ['BIKE', [149, 129, 99], [129, 99, 149]], ['BIKE', [221, 253, 247], [221, 247, 253]], ['BIKE', [272, 260, 266], [272, 266, 260]], ['BIKE', [78, 106, 189], [106, 78, 189]], ['BIKE', [86, 115, 178], [178, 86, 115]], ['BIKE', [120, 210, 245, 215], [210, 245, 120, 215]], ['BIKE', [26, 88, 83], [88, 26, 83]], ['BIKE', [224, 176, 282], [176, 282, 224]], ['BIKE', [136, 235, 298], [136, 235, 298]], ['BIKE', [207, 236, 264], [207, 236, 264]], ['BIKE', [180, 296], [296, 180]], ['BIKE', [155, 254, 233], [155, 233, 254]], ['BIKE', [80, 145, 97], [80, 97, 145]], ['BIKE', [8, 112, 96], [8, 112, 96]], ['BIKE', [292, 249], [249, 292]], ['BIKE', [186, 113], [113, 186]], ['BIKE', [95, 28, 165, 152], [28, 95, 165, 152]], ['BIKE', [49, 81], [81, 49]], ['BIKE', [259, 290, 270], [270, 290, 259]], ['BIKE', [181, 238, 227, 211], [211, 238, 227, 181]], ['BIKE', [144, 107, 154], [144, 154, 107]], ['BIKE', [11, 74, 65, 241], [11, 74, 65, 241]], ['BIKE', [94, 70, 77], [70, 94, 77]], ['BIKE', [117, 48], [117, 48]], ['BIKE', [56, 62, 98], [62, 56, 98]], ['CAR', [137, 185], [137, 185]], ['CAR', [0, 22, 66], [0, 22, 66]], ['CAR', [163, 257], [257, 163]], ['CAR', [23, 15], [23, 15]], ['CAR', [141, 299], [141, 299]], ['CAR', [79, 102], [102, 79]], ['CAR', [24, 89, 194], [24, 89, 194]], ['CAR', [197, 173], [173, 197]], ['CAR', [172, 82], [82, 172]], ['CAR', [41, 91], [91, 41]], ['CAR', [18, 55], [18, 55]], ['CAR', [226, 84], [84, 226]], ['CAR', [47, 63], [63, 47]], ['CAR', [9, 61], [9, 61]], ['CAR', [100, 139], [100, 139]], ['CAR', [35, 44, 76], [44, 35, 76]], ['CAR', [167, 193], [167, 193]], ['CAR', [140, 142, 287], [140, 142, 287]], ['CAR', [204, 230], [230, 204]], ['CAR', [184, 291], [184, 291]], ['CAR', [21, 85], [21, 85]], ['CAR', [190, 171], [171, 190]], ['CAR', [209, 286], [286, 209]], ['CAR', [294], [294]], ['CAR', [160], [160]], ['CAR', [191, 213], [213, 191]], ['CAR', [27, 108], [108, 27]], ['CAR', [71, 244], [71, 244]], ['CAR', [234, 284, 273], [284, 234, 273]], ['CAR', [146, 218], [218, 146]], ['CAR', [168, 262], [262, 168]], ['CAR', [6, 50], [6, 50]], ['CAR', [188, 232], [188, 232]], ['CAR', [179, 177], [179, 177]], ['CAR', [166, 212], [212, 166]], ['CAR', [19, 72, 38], [72, 19, 38]], ['CAR', [13, 127], [13, 127]], ['CAR', [156, 214], [156, 214]], ['CAR', [31, 90, 40], [31, 40, 90]], ['CAR', [271], [271]], ['CAR', [121], [121]], ['CAR', [237], [237]], ['CAR', [3, 34], [3, 34]], ['CAR', [25], [25]], ['CAR', [289, 243], [243, 289]], ['CAR', [278, 148], [148, 278]], ['CAR', [159, 208], [159, 208]], ['CAR', [30, 101], [30, 101]], ['CAR', [276, 258], [276, 258]], ['CAR', [42], [42]], ['CAR', [130, 275], [130, 275]], ['CAR', [196], [196]], ['CAR', [51], [51]], ['CAR', [7], [7]], ['CAR', [182], [182]], ['CAR', [33], [33]], ['CAR', [267, 162, 297], [297, 267, 162]], ['CAR', [75], [75]], ['CAR', [58], [58]], ['CAR', [281], [281]], ['CAR', [46], [46]], ['CAR', [69], [69]], ['CAR', [203, 195], [203, 195]], ['CAR', [240], [240]], ['CAR', [256], [256]]]

In [36]:
best_obj = 3777.42

In [37]:
def create_bundles_from_solution(solution, all_orders, all_riders, dist_mat, K):
    bundles = []
    
    for rider_type, shop_seq, dlv_seq in solution:
        # 해당 라이더 타입에 맞는 라이더 객체 선택
        rider = next((r for r in all_riders if r.type == rider_type), None)
        if rider is None:
            continue  # 해당 타입의 라이더가 없는 경우, 다음으로 넘어감
        
        # 번들에 포함된 주문의 총 부피 계산
        total_volume = get_total_volume(all_orders, shop_seq)
        
        # 번들의 총 거리를 계산
        total_dist = get_total_distance(K, dist_mat, shop_seq, dlv_seq)
        
        # 새로운 Bundle 객체 생성
        new_bundle = Bundle(all_orders, rider, shop_seq, dlv_seq, total_volume, total_dist)
        bundles.append(new_bundle)
    
    return bundles

# best_solution을 사용하여 Bundle 객체 리스트 생성
current_solution = create_bundles_from_solution(best_solution, ALL_ORDERS, ALL_RIDERS, DIST, K)

# 생성된 번들 리스트 확인
for bundle in current_solution:
    print(bundle)


Bundle(all_orders, BIKE, [14, 125, 105], [14, 125, 105], 73, True)
Bundle(all_orders, BIKE, [265, 293], [293, 265], 71, True)
Bundle(all_orders, BIKE, [274, 279, 219], [219, 274, 279], 100, True)
Bundle(all_orders, BIKE, [192, 175, 201], [175, 192, 201], 82, True)
Bundle(all_orders, BIKE, [268, 187, 202], [202, 187, 268], 74, True)
Bundle(all_orders, BIKE, [143, 223, 150], [150, 143, 223], 93, True)
Bundle(all_orders, BIKE, [123, 242, 147, 206], [147, 242, 206, 123], 91, True)
Bundle(all_orders, BIKE, [134, 246], [134, 246], 86, True)
Bundle(all_orders, BIKE, [161, 135, 239, 198], [135, 239, 198, 161], 89, True)
Bundle(all_orders, BIKE, [131, 250, 248], [248, 131, 250], 81, True)
Bundle(all_orders, BIKE, [157, 158, 170], [158, 170, 157], 64, True)
Bundle(all_orders, BIKE, [153, 124, 225, 109], [124, 153, 225, 109], 52, True)
Bundle(all_orders, BIKE, [133, 220, 217], [217, 220, 133], 75, True)
Bundle(all_orders, BIKE, [122, 169, 263], [263, 169, 122], 48, True)
Bundle(all_orders, BIKE, 

In [38]:
current_solution = create_bundles_from_solution(best_solution, ALL_ORDERS, ALL_RIDERS, DIST, K)

# 생성된 번들 리스트 확인
for bundle in current_solution:
    print(bundle)
    print(type(bundle))  # 각 요소의 타입을 출력


Bundle(all_orders, BIKE, [14, 125, 105], [14, 125, 105], 73, True)
<class 'util.Bundle'>
Bundle(all_orders, BIKE, [265, 293], [293, 265], 71, True)
<class 'util.Bundle'>
Bundle(all_orders, BIKE, [274, 279, 219], [219, 274, 279], 100, True)
<class 'util.Bundle'>
Bundle(all_orders, BIKE, [192, 175, 201], [175, 192, 201], 82, True)
<class 'util.Bundle'>
Bundle(all_orders, BIKE, [268, 187, 202], [202, 187, 268], 74, True)
<class 'util.Bundle'>
Bundle(all_orders, BIKE, [143, 223, 150], [150, 143, 223], 93, True)
<class 'util.Bundle'>
Bundle(all_orders, BIKE, [123, 242, 147, 206], [147, 242, 206, 123], 91, True)
<class 'util.Bundle'>
Bundle(all_orders, BIKE, [134, 246], [134, 246], 86, True)
<class 'util.Bundle'>
Bundle(all_orders, BIKE, [161, 135, 239, 198], [135, 239, 198, 161], 89, True)
<class 'util.Bundle'>
Bundle(all_orders, BIKE, [131, 250, 248], [248, 131, 250], 81, True)
<class 'util.Bundle'>
Bundle(all_orders, BIKE, [157, 158, 170], [158, 170, 157], 64, True)
<class 'util.Bundle'>


In [39]:
def generate_neighbors(current_solution, all_orders, all_riders, dist_mat, K):
    neighbors = []

    # 번들 간의 주문 교환 시도
    for i in range(len(current_solution)):
        for j in range(i + 1, len(current_solution)):
            bundle1 = current_solution[i]
            bundle2 = current_solution[j]

            for order1 in bundle1.shop_seq:
                for order2 in bundle2.shop_seq:
                    # 번들 간 주문 교환
                    new_bundle1_shop_seq = bundle1.shop_seq.copy()
                    new_bundle2_shop_seq = bundle2.shop_seq.copy()

                    # 교환 적용
                    new_bundle1_shop_seq.remove(order1)
                    new_bundle1_shop_seq.append(order2)

                    new_bundle2_shop_seq.remove(order2)
                    new_bundle2_shop_seq.append(order1)

                    # 새로운 배달 순서 생성 (예시: 기한을 기준으로 정렬)
                    new_bundle1_dlv_seq = sorted(new_bundle1_shop_seq, key=lambda order_id: all_orders[order_id].deadline)
                    new_bundle2_dlv_seq = sorted(new_bundle2_shop_seq, key=lambda order_id: all_orders[order_id].deadline)

                    # 새 번들에 포함된 주문의 총 부피 계산
                    new_bundle1_volume = get_total_volume(all_orders, new_bundle1_shop_seq)
                    new_bundle2_volume = get_total_volume(all_orders, new_bundle2_shop_seq)

                    # 새로운 번들의 총 거리를 계산
                    new_bundle1_total_dist = get_total_distance(K, dist_mat, new_bundle1_shop_seq, new_bundle1_dlv_seq)
                    new_bundle2_total_dist = get_total_distance(K, dist_mat, new_bundle2_shop_seq, new_bundle2_dlv_seq)

                    # 유효성 검증
                    feasible1 = test_route_feasibility(all_orders, bundle1.rider, new_bundle1_shop_seq, new_bundle1_dlv_seq)
                    feasible2 = test_route_feasibility(all_orders, bundle2.rider, new_bundle2_shop_seq, new_bundle2_dlv_seq)

                    if feasible1 == 0 and feasible2 == 0:
                        # 새로운 유효한 이웃 생성
                        new_solution = copy.deepcopy(current_solution)
                        new_solution[i] = Bundle(all_orders, bundle1.rider, new_bundle1_shop_seq, new_bundle1_dlv_seq, new_bundle1_volume, new_bundle1_total_dist)
                        new_solution[j] = Bundle(all_orders, bundle2.rider, new_bundle2_shop_seq, new_bundle2_dlv_seq, new_bundle2_volume, new_bundle2_total_dist)

                        neighbors.append(new_solution)

    return neighbors

generate_neighbors(current_solution, ALL_ORDERS, ALL_RIDERS, DIST, K)

NameError: name 'copy' is not defined

In [ ]:
import copy


def generate_neighbors(current_solution, all_orders, all_riders, dist_mat, K):
    neighbors = []

    # 번들 간의 주문 교환 시도
    for i in range(len(current_solution)):
        for j in range(i + 1, len(current_solution)):
            bundle1 = current_solution[i]
            bundle2 = current_solution[j]

            # 타입 확인
            # print(f"bundle1 type: {type(bundle1)}, bundle2 type: {type(bundle2)}")
            # print(f"bundle1 shop_seq: {bundle1.shop_seq}, bundle2 shop_seq: {bundle2.shop_seq}")

            for order1 in bundle1.shop_seq:
                for order2 in bundle2.shop_seq:
                    # 번들 간 주문 교환
                    new_bundle1_shop_seq = bundle1.shop_seq.copy()
                    new_bundle2_shop_seq = bundle2.shop_seq.copy()

                    # 교환 적용
                    new_bundle1_shop_seq.remove(order1)
                    new_bundle1_shop_seq.append(order2)

                    new_bundle2_shop_seq.remove(order2)
                    new_bundle2_shop_seq.append(order1)

                    # 새로운 배달 순서 생성 (예시: 기한을 기준으로 정렬)
                    new_bundle1_dlv_seq = sorted(new_bundle1_shop_seq, key=lambda order_id: all_orders[order_id].deadline)
                    new_bundle2_dlv_seq = sorted(new_bundle2_shop_seq, key=lambda order_id: all_orders[order_id].deadline)

                    # 새 번들에 포함된 주문의 총 부피 계산
                    new_bundle1_volume = get_total_volume(all_orders, new_bundle1_shop_seq)
                    new_bundle2_volume = get_total_volume(all_orders, new_bundle2_shop_seq)

                    # 새로운 번들의 총 거리를 계산
                    new_bundle1_total_dist = get_total_distance(K, dist_mat, new_bundle1_shop_seq, new_bundle1_dlv_seq)
                    new_bundle2_total_dist = get_total_distance(K, dist_mat, new_bundle2_shop_seq, new_bundle2_dlv_seq)

                    # 유효성 검증
                    feasible1 = test_route_feasibility(all_orders, bundle1.rider, new_bundle1_shop_seq, new_bundle1_dlv_seq)
                    feasible2 = test_route_feasibility(all_orders, bundle2.rider, new_bundle2_shop_seq, new_bundle2_dlv_seq)

                    if feasible1 == 0 and feasible2 == 0:
                        # 새로운 유효한 이웃 생성
                        new_solution = copy.deepcopy(current_solution)
                        new_solution[i] = Bundle(all_orders, bundle1.rider, new_bundle1_shop_seq, new_bundle1_dlv_seq, new_bundle1_volume, new_bundle1_total_dist)
                        new_solution[j] = Bundle(all_orders, bundle2.rider, new_bundle2_shop_seq, new_bundle2_dlv_seq, new_bundle2_volume, new_bundle2_total_dist)

                        # 새로운 solution을 원하는 형태로 변환
                        formatted_solution = [
                            [bundle.rider.type, bundle.shop_seq, bundle.dlv_seq]
                            for bundle in new_solution
                        ]

                        neighbors.append(formatted_solution)

    return neighbors

generate_neighbors(current_solution, ALL_ORDERS, ALL_RIDERS, DIST, K)

[[['BIKE', [14, 125, 105], [14, 125, 105]],
  ['BIKE', [265, 293], [293, 265]],
  ['BIKE', [274, 219, 249], [219, 249, 274]],
  ['BIKE', [192, 175, 201], [175, 192, 201]],
  ['BIKE', [268, 187, 202], [202, 187, 268]],
  ['BIKE', [143, 223, 150], [150, 143, 223]],
  ['BIKE', [123, 242, 147, 206], [147, 242, 206, 123]],
  ['BIKE', [134, 246], [134, 246]],
  ['BIKE', [161, 135, 239, 198], [135, 239, 198, 161]],
  ['BIKE', [131, 250, 248], [248, 131, 250]],
  ['BIKE', [157, 158, 170], [158, 170, 157]],
  ['BIKE', [153, 124, 225, 109], [124, 153, 225, 109]],
  ['BIKE', [133, 220, 217], [217, 220, 133]],
  ['BIKE', [122, 169, 263], [263, 169, 122]],
  ['BIKE', [20, 52], [52, 20]],
  ['BIKE', [29, 32, 45], [32, 45, 29]],
  ['BIKE', [64, 43, 12], [43, 12, 64]],
  ['BIKE', [5, 10, 57, 87], [10, 5, 57, 87]],
  ['BIKE', [116, 205, 104], [116, 205, 104]],
  ['BIKE', [59, 174], [59, 174]],
  ['BIKE', [216, 183, 280, 277], [183, 216, 277, 280]],
  ['BIKE', [36, 4, 114], [4, 114, 36]],
  ['BIKE', [28

In [ ]:
import numpy as np
import time
import itertools
import random
from itertools import permutations
from util import Bundle, Order, Rider, select_two_bundles, try_merging_bundles, get_total_distance, get_total_volume, test_route_feasibility, get_cheaper_available_riders, try_bundle_rider_changing
import concurrent.futures
import json

In [ ]:
problem_file = 'STAGE1_1.json'
# problem_file = "TEST_K50_1.json"
timelimit = 60

# np.random.seed(1)

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time)

In [ ]:
import numpy as np
import time
import itertools
import random
from itertools import permutations
from util import Bundle, select_two_bundles, try_merging_bundles, get_total_distance, get_total_volume, test_route_feasibility, get_cheaper_available_riders, try_bundle_rider_changing
import concurrent.futures

def regret_order_clustering(K, dist_mat, all_orders, all_riders):
    # 차량(Car) 배달원 리스트 생성
    car_riders = [r for r in all_riders if r.type == 'CAR']

    unassigned_orders = set(range(K))
    bundles = []

    while unassigned_orders:
        best_cluster = None
        best_regret_value = float('-inf')

        # 가능한 모든 주문 묶음 조합을 생성
        for cluster_size in range(2, len(unassigned_orders) + 1):
            for cluster in itertools.combinations(unassigned_orders, cluster_size):
                best_cost = float('inf')
                second_best_cost = float('inf')
                best_rider = None

                # 각 묶음에 대해 최선과 두 번째로 좋은 비용 계산
                for rider in car_riders:  # 이제 오직 차량(Car) 배달원에 대해서만 반복합니다
                    # 차량(Car) 배달원의 용량을 기준으로 묶음 가능 여부 확인
                    if sum(all_orders[i].volume for i in cluster) <= rider.capa:
                        for shop_seq in itertools.permutations(cluster):
                            for dlv_seq in itertools.permutations(cluster):
                                if test_route_feasibility(all_orders, rider, shop_seq, dlv_seq) == 0:
                                    cost = get_total_distance(K, dist_mat, shop_seq, dlv_seq)
                                    if cost < best_cost:
                                        second_best_cost = best_cost
                                        best_cost = cost
                                        best_rider = rider
                                    elif cost < second_best_cost:
                                        second_best_cost = cost

                # Regret 값 계산
                regret_value = second_best_cost - best_cost
                if regret_value > best_regret_value:
                    best_regret_value = regret_value
                    best_cluster = (cluster, best_rider, best_cost, shop_seq, dlv_seq)

        if best_cluster:
            cluster, rider, cost, shop_seq, dlv_seq = best_cluster
            total_volume = sum(all_orders[i].volume for i in cluster)
            total_dist = get_total_distance(K, dist_mat, shop_seq, dlv_seq)
            bundles.append(Bundle(all_orders, rider, list(shop_seq), list(dlv_seq), total_volume, total_dist))
            unassigned_orders -= set(cluster)
            rider.available_number -= 1

    return bundles

In [ ]:
def find_nearest_orders(current_bundle, remaining_orders, dist_mat, K, num_orders):
    bundle_size = len(current_bundle)
    
    distances = []
    for order in remaining_orders:
        total_distance = 0
        for existing_order in current_bundle:
            pickup_distance = dist_mat[existing_order.id, order.id]
            delivery_distance = dist_mat[existing_order.id + K, order.id + K]
            total_distance += (pickup_distance + delivery_distance)
        average_distance = total_distance / bundle_size
        distances.append((order, average_distance))
    
    distances.sort(key=lambda x: x[1])
    nearest_orders = [order for order, _ in distances[:num_orders]]
    return nearest_orders

In [ ]:
def cluster_orders(K, dist_mat, all_orders, cluster_size):
    unassigned_orders = list(range(K))
    clusters = []

    while len(unassigned_orders) > 0:
        current_order = unassigned_orders.pop(0)
        current_bundle = [all_orders[current_order]]
        
        # 가까운 주문들로 클러스터 생성
        nearest_orders = find_nearest_orders(current_bundle, [all_orders[i] for i in unassigned_orders], dist_mat, K, cluster_size - 1)
        cluster = [current_order] + [order.id for order in nearest_orders]
        
        clusters.append(cluster)
        
        # 클러스터링된 주문들을 unassigned_orders에서 제거
        unassigned_orders = [order for order in unassigned_orders if order not in cluster]
        
    return clusters

In [ ]:
def regret_order_clustering_with_preclustering(K, dist_mat, all_orders, all_riders, cluster_size, max_permutations=50):
    # 차량(Car) 배달원 리스트 생성
    car_riders = [r for r in all_riders if r.type == 'CAR']
    
    # 주문을 클러스터링
    clusters = cluster_orders(K, dist_mat, all_orders, cluster_size)

    bundles = []
    assigned_orders = set()  # 이미 번들에 할당된 주문을 추적하기 위해 사용

    for cluster in clusters:
        unassigned_orders = set(cluster) - assigned_orders  # 이미 할당된 주문은 제외
        
        while unassigned_orders:
            best_cluster = None
            best_regret_value = float('-inf')

            # 가능한 모든 주문 묶음 조합을 생성
            for bundle_size in range(2, min(6, len(unassigned_orders) + 1)):  # 작은 크기의 묶음부터 탐색
                for cluster_subset in itertools.combinations(unassigned_orders, bundle_size):
                    best_cost = float('inf')
                    second_best_cost = float('inf')
                    best_rider = None

                    # 각 묶음에 대해 최선과 두 번째로 좋은 비용 계산
                    for rider in car_riders:
                        # 차량(Car) 배달원의 용량을 기준으로 묶음 가능 여부 확인
                        if sum(all_orders[i].volume for i in cluster_subset) <= rider.capa:
                            permuted_sequences = itertools.islice(itertools.permutations(cluster_subset), max_permutations)
                            for shop_seq in permuted_sequences:
                                for dlv_seq in itertools.permutations(shop_seq):
                                    if test_route_feasibility(all_orders, rider, shop_seq, dlv_seq) == 0:
                                        total_distance = get_total_distance(K, dist_mat, shop_seq, dlv_seq)
                                        total_cost = rider.fixed_cost + (total_distance / 100.0) * rider.var_cost
                                        avg_cost = total_cost / len(cluster_subset)
                                        if avg_cost < best_cost:
                                            second_best_cost = best_cost
                                            best_cost = avg_cost
                                            best_rider = rider
                                        elif avg_cost < second_best_cost:
                                            second_best_cost = avg_cost

                    # Regret 값 계산
                    regret_value = second_best_cost - best_cost
                    if regret_value > best_regret_value and best_rider is not None:
                        best_regret_value = regret_value
                        best_cluster = (cluster_subset, best_rider, best_cost, shop_seq, dlv_seq)

            if best_cluster:
                cluster_subset, rider, cost, shop_seq, dlv_seq = best_cluster
                total_volume = sum(all_orders[i].volume for i in cluster_subset)
                total_dist = get_total_distance(K, dist_mat, shop_seq, dlv_seq)
                bundles.append(Bundle(all_orders, rider, list(shop_seq), list(dlv_seq), total_volume, total_dist))
                unassigned_orders -= set(cluster_subset)
                assigned_orders.update(cluster_subset)  # 이미 할당된 주문을 기록
                rider.available_number -= 1
            else:
                # 어떤 주문도 묶이지 않을 경우, 첫 번째 주문을 단독으로 묶음으로 만들어 추가
                order_id = unassigned_orders.pop()
                bundles.append(Bundle(all_orders, car_riders[0], [order_id], [order_id], all_orders[order_id].volume, 0))
                assigned_orders.add(order_id)

    # 모든 주문이 할당되었는지 확인
    if len(assigned_orders) != K:
        raise ValueError(f"Some orders were not assigned to any bundle: {set(range(K)) - assigned_orders}")

    # 평균 비용 계산 및 출력
    total_cost = sum(bundle.rider.fixed_cost + (bundle.total_dist / 100.0) * bundle.rider.var_cost for bundle in bundles)
    avg_cost = total_cost / len(bundles)
    print(f'Average cost per bundle: {avg_cost}')

    return bundles




In [ ]:
regret_order_clustering_with_preclustering(K, DIST, ALL_ORDERS, ALL_RIDERS, cluster_size=10)

Average cost per bundle: 8763.883333333333


[Bundle(all_orders, CAR, [10, 58], [58, 10], 37, True),
 Bundle(all_orders, CAR, [88, 72], [72, 88], 55, True),
 Bundle(all_orders, CAR, [11, 32], [32, 11], 54, True),
 Bundle(all_orders, CAR, [90, 84, 78], [78, 84, 90], 107, True),
 Bundle(all_orders, CAR, [0], [0], 21, True),
 Bundle(all_orders, CAR, [41, 1], [1, 41], 33, True),
 Bundle(all_orders, CAR, [68, 98], [98, 68], 51, True),
 Bundle(all_orders, CAR, [61, 74], [74, 61], 35, True),
 Bundle(all_orders, CAR, [85, 79], [79, 85], 61, True),
 Bundle(all_orders, CAR, [96], [96], 28, True),
 Bundle(all_orders, CAR, [29], [29], 36, True),
 Bundle(all_orders, CAR, [39, 2], [2, 39], 37, True),
 Bundle(all_orders, CAR, [48, 77], [77, 48], 48, True),
 Bundle(all_orders, CAR, [18, 17], [17, 18], 34, True),
 Bundle(all_orders, CAR, [55, 50], [50, 55], 39, True),
 Bundle(all_orders, CAR, [80], [80], 13, True),
 Bundle(all_orders, CAR, [25], [25], 39, True),
 Bundle(all_orders, CAR, [70, 65], [65, 70], 41, True),
 Bundle(all_orders, CAR, [60,

In [ ]:
import numpy as np
import time
import itertools
import random
from itertools import permutations
from util import Bundle, Order, Rider, select_two_bundles, try_merging_bundles, get_total_distance, get_total_volume, test_route_feasibility, get_cheaper_available_riders, try_bundle_rider_changing
import concurrent.futures
import json

In [ ]:
import numpy as np
import time
import itertools
import random
from itertools import permutations
from util import Bundle, Order, Rider, select_two_bundles, try_merging_bundles, get_total_distance, get_total_volume, test_route_feasibility, get_cheaper_available_riders, try_bundle_rider_changing
import concurrent.futures
import json

problem_file = 'STAGE1_1.json'
# problem_file = "TEST_K50_1.json"
timelimit = 60

# np.random.seed(1)

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time)

In [ ]:
def calculate_efficiencies(K, all_riders, all_orders, dist_mat):
    # 모든 주문의 부피 평균 계산
    order_volumes = [order.volume for order in all_orders]
    avg_volume = np.mean(order_volumes)

    # d1, d2, d3 계산
    d1 = np.sum(dist_mat[:K, :K]) / (K*K)
    d2 = np.sum(dist_mat[:K, K:2*K]) / (K*K)
    d3 = np.sum(dist_mat[K:2*K, K:2*K]) / (K*K)

    # 각 배달원의 효율성 지표 계산 함수
    def calculate_efficiency(rider, avg_volume, d1, d2, d3):
        capacity = rider.capa
        variable_cost = rider.var_cost
        fixed_cost = rider.fixed_cost
        
        Ri = capacity / avg_volume
        Xi = (Ri - 1) * d1 + (Ri - 1) * d3 + d2
        efficiency = fixed_cost + (Xi / 100) * variable_cost
        
        return efficiency

    # 각 배달원의 효율성 지표 계산
    efficiencies = []
    for rider in all_riders:
        rider_type = rider.type
        efficiency = calculate_efficiency(rider, avg_volume, d1, d2, d3)
        efficiencies.append([rider_type, efficiency])

    # 효율성 지표 반환
    return efficiencies

In [ ]:
def find_nearest_orders(current_bundle, remaining_orders, dist_mat, K, num_orders):
    bundle_size = len(current_bundle)
    
    distances = []
    for order in remaining_orders:
        total_distance = 0
        for existing_order in current_bundle:
            pickup_distance = dist_mat[existing_order.id, order.id]
            delivery_distance = dist_mat[existing_order.id + K, order.id + K]
            total_distance += (pickup_distance + delivery_distance)
        average_distance = total_distance / bundle_size
        distances.append((order, average_distance))
    
    distances.sort(key=lambda x: x[1])
    nearest_orders = [order for order, _ in distances[:num_orders]]
    return nearest_orders

In [ ]:
def assign_orders_to_rider(rider, orders, dist_mat, K, all_orders):
    bundles = []
    remaining_orders = orders[:]
    
    while remaining_orders and rider.available_number > 0:
        current_order = remaining_orders.pop(0)
        current_bundle = [current_order]
        shop_seq = [current_order.id]
        delivery_seq = sorted(shop_seq, key=lambda order_id: all_orders[order_id].deadline)
        
        is_feasible = test_route_feasibility(all_orders, rider, shop_seq, delivery_seq)
        if is_feasible != 0:
            remaining_orders.insert(0, current_order)
            return bundles, remaining_orders
        
        current_volume = current_order.volume
        current_time = current_order.ready_time

        while True:
            nearest_orders = find_nearest_orders(current_bundle, remaining_orders, dist_mat, K, 30)
            added = False

            if len(current_bundle) >= 4:
                break
            
            for next_order in nearest_orders:
                if current_volume + next_order.volume > rider.capa:
                    continue
                
                current_bundle_ids = [o.id for o in current_bundle]
                next_bundle_ids = [next_order.id]

                combined_ids = current_bundle_ids + next_bundle_ids
                pickup_permutations = itertools.permutations(combined_ids)

                valid_combinations = []
                
                for perm_shop_seq in pickup_permutations:
                    delivery_permutations = itertools.permutations(perm_shop_seq)
                    for perm_dlv_seq in delivery_permutations:
                        new_bundle = Bundle(all_orders, rider, list(perm_shop_seq), list(perm_dlv_seq), current_volume + next_order.volume, 0)
                        new_bundle.total_dist = get_total_distance(K, dist_mat, list(perm_shop_seq), list(perm_dlv_seq))
                        new_bundle.update_cost()

                        is_feasible = test_route_feasibility(all_orders, rider, list(perm_shop_seq), list(perm_dlv_seq))
                        if is_feasible == 0:
                            valid_combinations.append((list(perm_shop_seq), list(perm_dlv_seq)))

                if valid_combinations:
                    best_combination = min(valid_combinations, key=lambda x: get_total_distance(K, dist_mat, x[0], x[1]))
                    best_shop_seq, best_dlv_seq = best_combination

                    current_bundle.append(next_order)
                    current_volume += next_order.volume
                    current_time += rider.T[current_bundle[-2].id, next_order.id]
                    remaining_orders.remove(next_order)
                    added = True

                    # 선택된 best_shop_seq와 best_dlv_seq로 번들 갱신
                    shop_seq = best_shop_seq
                    delivery_seq = best_dlv_seq
                    break

            if not added:
                break

        final_bundle = Bundle(all_orders, rider, shop_seq, delivery_seq, current_volume, get_total_distance(K, dist_mat, shop_seq, delivery_seq))
        bundles.append(final_bundle)
        rider.available_number -= 1

    return bundles, remaining_orders

In [ ]:
def assign_riders_with_weighted_probability(riders, effectiveness_indicator):
    total_effectiveness = sum([indicator[1] for indicator in effectiveness_indicator])
    weights = [total_effectiveness / indicator[1] for indicator in effectiveness_indicator]
    selected_rider = random.choices(riders, weights=weights, k=1)[0]
    return selected_rider

In [ ]:
def optimize_single_order_bundles(best_solution, all_orders, all_riders, dist_mat, K, max_nearest_bundles=5):
    single_order_bundles = [bundle for bundle in best_solution if len(bundle.shop_seq) == 1]
    optimized_bundles = []
    remaining_solution = best_solution[:]  # best_solution을 복사하여 사용

    for single_bundle in single_order_bundles:
        single_order_id = single_bundle.shop_seq[0]
        single_order = all_orders[single_order_id]

        # 해당 주문에 가장 가까운 5개의 주문 찾기
        nearest_orders = find_nearest_orders([single_order], all_orders, dist_mat, K, max_nearest_bundles)
        nearest_order_ids = [order.id for order in nearest_orders]

        # 이 주문들이 들어가 있는 번들들 찾기
        associated_bundles = []
        for order_id in nearest_order_ids:
            for bundle in remaining_solution:  # 남아 있는 솔루션에서 번들 찾기
                if order_id in bundle.shop_seq:
                    associated_bundles.append(bundle)
                    break

        # 현재 번들과 찾은 번들을 합침
        candidate_bundles = [single_bundle] + associated_bundles
        candidate_order_ids = [order_id for bundle in candidate_bundles for order_id in bundle.shop_seq]

        # best_solution에서 candidate_bundles에 있는 번들을 제거
        remaining_solution = [bundle for bundle in remaining_solution if bundle not in candidate_bundles]

        # 라이더 선택 및 할당
        rider = assign_riders_with_weighted_probability(all_riders, calculate_efficiencies(K, all_riders, all_orders, dist_mat))
        if rider.available_number > 0:
            new_rider = rider
        else:
            continue  # 선택된 라이더가 없거나, 할당 가능한 라이더가 없는 경우 건너뜀

        # 번들로 다시 묶기
        new_bundles, remaining_orders = assign_orders_to_rider(new_rider, [all_orders[i] for i in candidate_order_ids], dist_mat, K, all_orders)

        # 기존 번들들과 새로운 번들들 간의 비용 비교
        existing_total_cost = sum(bundle.cost for bundle in candidate_bundles)
        new_total_cost = sum(bundle.cost for bundle in new_bundles)

        if new_total_cost < existing_total_cost:
            optimized_bundles.extend(new_bundles)
        else:
            optimized_bundles.extend(candidate_bundles)

    # 최종 솔루션으로 업데이트
    final_solution = remaining_solution + optimized_bundles

    return final_solution


In [ ]:
def single_run_algorithm(K, all_orders, all_riders, dist_mat, timelimit=60):
    start_time = time.time()

    for r in all_riders:
        r.T = np.round(dist_mat / r.speed + r.service_time)

    # 효율성 지표 계산
    effectiveness_indicator = calculate_efficiencies(K, all_riders, all_orders, dist_mat)
    effectiveness_dict = {rider.type: effectiveness for rider, effectiveness in zip(all_riders, effectiveness_indicator)}

    # 주문들을 무작위로 섞기
    sorted_orders = random.sample(all_orders, len(all_orders))

    # 모든 라이더를 합쳐서 처리
    all_riders_list = all_riders

    all_bundles = []

    while sorted_orders and all_riders_list:
        rider = assign_riders_with_weighted_probability(all_riders_list, effectiveness_indicator)
        if rider.available_number > 0:
            bundles, sorted_orders = assign_orders_to_rider(rider, sorted_orders, dist_mat, K, all_orders)
            for bundle in bundles:
                all_bundles.append(bundle)

    best_obj = sum((bundle.cost for bundle in all_bundles)) / K
    print(f'Initial best obj = {best_obj}')

    return all_bundles

In [ ]:
best_solution = single_run_algorithm(K, ALL_ORDERS, ALL_RIDERS, DIST, 60)
best_solution

Initial best obj = 4213.42


[Bundle(all_orders, CAR, [26, 57], [26, 57], 114, True),
 Bundle(all_orders, CAR, [30, 63], [30, 63], 38, True),
 Bundle(all_orders, CAR, [72, 94, 88], [72, 88, 94], 83, True),
 Bundle(all_orders, CAR, [91, 99], [99, 91], 37, True),
 Bundle(all_orders, CAR, [52, 50, 66], [52, 50, 66], 56, True),
 Bundle(all_orders, CAR, [42, 53, 35], [35, 53, 42], 59, True),
 Bundle(all_orders, CAR, [3], [3], 44, True),
 Bundle(all_orders, CAR, [15, 23, 34, 20], [23, 15, 20, 34], 90, True),
 Bundle(all_orders, CAR, [71, 97], [71, 97], 49, True),
 Bundle(all_orders, CAR, [74, 85, 98], [74, 98, 85], 75, True),
 Bundle(all_orders, CAR, [11, 28, 32], [11, 28, 32], 70, True),
 Bundle(all_orders, CAR, [51, 44, 36], [51, 44, 36], 85, True),
 Bundle(all_orders, CAR, [2, 39], [2, 39], 37, True),
 Bundle(all_orders, CAR, [84, 87], [87, 84], 93, True),
 Bundle(all_orders, CAR, [18, 25], [25, 18], 57, True),
 Bundle(all_orders, CAR, [19, 27], [19, 27], 56, True),
 Bundle(all_orders, CAR, [59, 76, 89], [59, 89, 76]

In [ ]:
single_order_bundles = [bundle for bundle in best_solution if len(bundle.shop_seq) == 1]
single_order_bundles

[Bundle(all_orders, CAR, [3], [3], 44, True),
 Bundle(all_orders, CAR, [95], [95], 13, True),
 Bundle(all_orders, CAR, [55], [55], 19, True),
 Bundle(all_orders, CAR, [78], [78], 36, True),
 Bundle(all_orders, CAR, [12], [12], 37, True)]

In [ ]:
for single_bundle in single_order_bundles:
        single_order_id = single_bundle.shop_seq[0]
        single_order = ALL_ORDERS[single_order_id]

single_order

Order([12, 371, 35.99940527853281, 125.97358456191878, 36.00234063853281, 125.96103896191876, 37, 300, 2440])

In [ ]:
nearest_orders = find_nearest_orders([single_order], ALL_ORDERS, DIST, K, 5)
nearest_orders

[Order([12, 371, 35.99940527853281, 125.97358456191878, 36.00234063853281, 125.96103896191876, 37, 300, 2440]),
 Order([57, 1801, 35.99937751853281, 125.97433449191878, 36.001320848532814, 125.96292794191878, 24, 1200, 4728]),
 Order([90, 3322, 35.99696112853281, 125.97402904191878, 36.00033715853281, 125.97511628191876, 16, 900, 5731]),
 Order([78, 2947, 36.00104393853281, 125.97094587191876, 35.99419507853281, 125.97397814191878, 36, 1800, 6395]),
 Order([67, 2402, 35.99718327853281, 125.97136261191878, 36.00379367853281, 125.97942532191878, 18, 600, 4724])]

In [ ]:
nearest_order_ids = [order.id for order in nearest_orders]

In [ ]:
optimized_bundles = []
remaining_solution = best_solution[:]

In [ ]:
associated_bundles = set()  # 중복을 피하기 위해 set 사용
for order_id in nearest_order_ids:
    for bundle in remaining_solution:  # 남아 있는 솔루션에서 번들 찾기
        if order_id in bundle.shop_seq:
            associated_bundles.add(bundle)
            break

associated_bundles = list(associated_bundles)

associated_bundles

[Bundle(all_orders, CAR, [12], [12], 37, True),
 Bundle(all_orders, CAR, [26, 57], [26, 57], 114, True),
 Bundle(all_orders, CAR, [78], [78], 36, True),
 Bundle(all_orders, CAR, [73, 67, 90], [67, 73, 90], 44, True)]

In [ ]:
candidate_bundles = associated_bundles[:]  # associated_bundles을 복사
if single_bundle not in associated_bundles:
    candidate_bundles.append(single_bundle)

candidate_bundles

[Bundle(all_orders, CAR, [12], [12], 37, True),
 Bundle(all_orders, CAR, [26, 57], [26, 57], 114, True),
 Bundle(all_orders, CAR, [78], [78], 36, True),
 Bundle(all_orders, CAR, [73, 67, 90], [67, 73, 90], 44, True)]

In [ ]:
 candidate_order_ids = [order_id for bundle in candidate_bundles for order_id in bundle.shop_seq]

In [ ]:
candidate_order_ids

[12, 26, 57, 78, 73, 67, 90]

In [ ]:
rider = assign_riders_with_weighted_probability(ALL_RIDERS, calculate_efficiencies(K, ALL_RIDERS, ALL_ORDERS, DIST))
if rider.available_number > 0:
    new_rider = rider

new_rider

Rider([WALK, 1.3227513227513228, 70, 30, 8000, 120, 27])

In [ ]:
new_bundles, remaining_orders = assign_orders_to_rider(new_rider, [ALL_ORDERS[i] for i in candidate_order_ids], DIST, K, ALL_ORDERS)

new_bundles

[Bundle(all_orders, CAR, [12], [12], 37, True),
 Bundle(all_orders, CAR, [26, 57], [26, 57], 114, True),
 Bundle(all_orders, CAR, [73, 73, 90, 90], [73, 73, 90, 90], 52, True),
 Bundle(all_orders, CAR, [67, 67], [67, 67], 36, True),
 Bundle(all_orders, CAR, [78], [78], 36, True)]

In [ ]:
import numpy as np
import time
import itertools
import random
from itertools import permutations
from util import Bundle, Order, Rider, select_two_bundles, try_merging_bundles, get_total_distance, get_total_volume, test_route_feasibility, get_cheaper_available_riders, try_bundle_rider_changing
import concurrent.futures
import json

problem_file = 'STAGE1_1.json'
# problem_file = "TEST_K50_1.json"
timelimit = 60

# np.random.seed(1)

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

all_orders = [Order(order_info) for order_info in prob['ORDERS']]
all_riders = [Rider(rider_info) for rider_info in prob['RIDERS']]

dist_mat = np.array(prob['DIST'])
for r in all_riders:
    r.T = np.round(dist_mat/r.speed + r.service_time)

In [ ]:
def calculate_efficiencies(K, all_riders, all_orders, dist_mat):
    # 모든 주문의 부피 평균 계산
    order_volumes = [order.volume for order in all_orders]
    avg_volume = np.mean(order_volumes)

    # d1, d2, d3 계산
    d1 = np.sum(dist_mat[:K, :K]) / (K*K)
    d2 = np.sum(dist_mat[:K, K:2*K]) / (K*K)
    d3 = np.sum(dist_mat[K:2*K, K:2*K]) / (K*K)

    # 각 배달원의 효율성 지표 계산 함수
    def calculate_efficiency(rider, avg_volume, d1, d2, d3):
        capacity = rider.capa
        variable_cost = rider.var_cost
        fixed_cost = rider.fixed_cost
        
        Ri = capacity / avg_volume
        Xi = (Ri - 1) * d1 + (Ri - 1) * d3 + d2
        efficiency = fixed_cost + (Xi / 100) * variable_cost
        
        return efficiency

    # 각 배달원의 효율성 지표 계산
    efficiencies = []
    for rider in all_riders:
        rider_type = rider.type
        efficiency = calculate_efficiency(rider, avg_volume, d1, d2, d3)
        efficiencies.append([rider_type, efficiency])

    # 효율성 지표 반환
    return efficiencies

In [ ]:
def find_nearest_orders(current_bundle, remaining_orders, dist_mat, K, num_orders):
    bundle_size = len(current_bundle)
    
    distances = []
    for order in remaining_orders:
        total_distance = 0
        for existing_order in current_bundle:
            pickup_distance = dist_mat[existing_order.id, order.id]
            delivery_distance = dist_mat[existing_order.id + K, order.id + K]
            total_distance += (pickup_distance + delivery_distance)
        average_distance = total_distance / bundle_size
        distances.append((order, average_distance))
    
    distances.sort(key=lambda x: x[1])
    nearest_orders = [order for order, _ in distances[:num_orders]]
    return nearest_orders

In [ ]:
def assign_orders_to_rider(rider, orders, dist_mat, K, all_orders):
    bundles = []
    remaining_orders = orders[:]
    
    while remaining_orders and rider.available_number > 0:
        current_order = remaining_orders.pop(0)
        current_bundle = [current_order]
        shop_seq = [current_order.id]
        delivery_seq = sorted(shop_seq, key=lambda order_id: all_orders[order_id].deadline)
        
        is_feasible = test_route_feasibility(all_orders, rider, shop_seq, delivery_seq)
        if is_feasible != 0:
            remaining_orders.insert(0, current_order)
            return bundles, remaining_orders
        
        current_volume = current_order.volume
        current_time = current_order.ready_time

        while True:
            nearest_orders = find_nearest_orders(current_bundle, remaining_orders, dist_mat, K, 30)
            added = False

            if len(current_bundle) >= 4:
                break
            
            for next_order in nearest_orders:
                if current_volume + next_order.volume > rider.capa:
                    continue
                
                current_bundle_ids = [o.id for o in current_bundle]
                next_bundle_ids = [next_order.id]

                combined_ids = current_bundle_ids + next_bundle_ids
                pickup_permutations = itertools.permutations(combined_ids)

                valid_combinations = []
                
                for perm_shop_seq in pickup_permutations:
                    delivery_permutations = itertools.permutations(perm_shop_seq)
                    for perm_dlv_seq in delivery_permutations:
                        new_bundle = Bundle(all_orders, rider, list(perm_shop_seq), list(perm_dlv_seq), current_volume + next_order.volume, 0)
                        new_bundle.total_dist = get_total_distance(K, dist_mat, list(perm_shop_seq), list(perm_dlv_seq))
                        new_bundle.update_cost()

                        is_feasible = test_route_feasibility(all_orders, rider, list(perm_shop_seq), list(perm_dlv_seq))
                        if is_feasible == 0:
                            valid_combinations.append((list(perm_shop_seq), list(perm_dlv_seq)))

                if valid_combinations:
                    best_combination = min(valid_combinations, key=lambda x: get_total_distance(K, dist_mat, x[0], x[1]))
                    best_shop_seq, best_dlv_seq = best_combination

                    current_bundle.append(next_order)
                    current_volume += next_order.volume
                    current_time += rider.T[current_bundle[-2].id, next_order.id]
                    remaining_orders.remove(next_order)
                    added = True

                    # 선택된 best_shop_seq와 best_dlv_seq로 번들 갱신
                    shop_seq = best_shop_seq
                    delivery_seq = best_dlv_seq
                    break

            if not added:
                break

        final_bundle = Bundle(all_orders, rider, shop_seq, delivery_seq, current_volume, get_total_distance(K, dist_mat, shop_seq, delivery_seq))
        bundles.append(final_bundle)
        rider.available_number -= 1

    return bundles, remaining_orders

In [ ]:
def assign_riders_with_weighted_probability(riders, effectiveness_indicator):
    total_effectiveness = sum([indicator[1] for indicator in effectiveness_indicator])
    weights = [total_effectiveness / indicator[1] for indicator in effectiveness_indicator]
    selected_rider = random.choices(riders, weights=weights, k=1)[0]
    return selected_rider

In [ ]:
def optimize_single_order_bundles(best_solution, all_orders, all_riders, dist_mat, K, max_nearest_bundles=5):
    single_order_bundles = [bundle for bundle in best_solution if len(bundle.shop_seq) == 1]
    optimized_bundles = []
    remaining_solution = best_solution[:]  # best_solution을 복사하여 사용

    for single_bundle in single_order_bundles:
        single_order_id = single_bundle.shop_seq[0]
        single_order = all_orders[single_order_id]

        # 해당 주문에 가장 가까운 5개의 주문 찾기
        nearest_orders = find_nearest_orders([single_order], all_orders, dist_mat, K, max_nearest_bundles)
        nearest_order_ids = [order.id for order in nearest_orders]

        # 이 주문들이 들어가 있는 번들들 찾기
        associated_bundles = set()  # 중복을 피하기 위해 set 사용
        for order_id in nearest_order_ids:
            for bundle in remaining_solution:  # 남아 있는 솔루션에서 번들 찾기
                if order_id in bundle.shop_seq:
                    associated_bundles.add(bundle)
                    break

        # 현재 번들과 찾은 번들을 합침 (단, single_bundle이 이미 associated_bundles에 있을 경우 제외)
        candidate_bundles = list(associated_bundles)  # set을 list로 변환
        if single_bundle not in associated_bundles:
            candidate_bundles.append(single_bundle)

        candidate_order_ids = [order_id for bundle in candidate_bundles for order_id in bundle.shop_seq]

        # best_solution에서 candidate_bundles에 있는 번들을 제거
        remaining_solution = [bundle for bundle in remaining_solution if bundle not in candidate_bundles]

        # 라이더 선택 및 할당
        while candidate_order_ids:  # 묶이지 않은 주문이 있을 때까지 반복
            rider = assign_riders_with_weighted_probability(all_riders, calculate_efficiencies(K, all_riders, all_orders, dist_mat))
            if rider.available_number > 0:
                new_rider = rider
            else:
                break  # 선택된 라이더가 없거나, 할당 가능한 라이더가 없는 경우 반복 종료

            # 번들로 다시 묶기
            new_bundles, remaining_orders = assign_orders_to_rider(new_rider, [all_orders[i] for i in candidate_order_ids], dist_mat, K, all_orders)

            # 기존 번들들과 새로운 번들들 간의 비용 비교
            existing_total_cost = sum(bundle.cost for bundle in candidate_bundles)
            new_total_cost = sum(bundle.cost for bundle in new_bundles)

            if new_total_cost < existing_total_cost:
                optimized_bundles.extend(new_bundles)
                candidate_order_ids = [order.id for order in remaining_orders]  # 남아 있는 주문 갱신
            else:
                optimized_bundles.extend(candidate_bundles)
                break  # 새로운 번들로 묶지 않는 것이 더 나은 경우 반복 종료

    # 최종 솔루션으로 업데이트
    final_solution = remaining_solution + optimized_bundles

    # 최종 솔루션의 목적함수 계산 (총 비용)
    final_solution_cost = sum(bundle.cost for bundle in final_solution)

    return final_solution

In [ ]:
def single_run_algorithm(K, all_orders, all_riders, dist_mat, timelimit=60):
    start_time = time.time()

    for r in all_riders:
        r.T = np.round(dist_mat / r.speed + r.service_time)

    # 효율성 지표 계산
    effectiveness_indicator = calculate_efficiencies(K, all_riders, all_orders, dist_mat)
    effectiveness_dict = {rider.type: effectiveness for rider, effectiveness in zip(all_riders, effectiveness_indicator)}

    # 주문들을 무작위로 섞기
    sorted_orders = random.sample(all_orders, len(all_orders))

    # 모든 라이더를 합쳐서 처리
    all_riders_list = all_riders

    all_bundles = []

    while sorted_orders and all_riders_list:
        rider = assign_riders_with_weighted_probability(all_riders_list, effectiveness_indicator)
        if rider.available_number > 0:
            bundles, sorted_orders = assign_orders_to_rider(rider, sorted_orders, dist_mat, K, all_orders)
            for bundle in bundles:
                all_bundles.append(bundle)

    best_obj = sum((bundle.cost for bundle in all_bundles)) / K
    print(f'Initial best obj = {best_obj}')

    return all_bundles

In [ ]:
def algorithm(K, all_orders, all_riders, dist_mat, timelimit=60, num_processes=30):
    with concurrent.futures.ProcessPoolExecutor(max_workers=num_processes) as executor:
        futures = [executor.submit(single_run_algorithm, K, all_orders, all_riders, dist_mat, timelimit) for _ in range(num_processes)]
        results = [future.result() for future in concurrent.futures.as_completed(futures)]

    # 각 프로세스의 목적함수 출력
    for i, result in enumerate(results):
        solution, obj_value = result
        print(f'Objective value from process {i+1}: {obj_value}')

    # 최적의 solution 선택
    best_solution = min(results, key=lambda x: x[1])[0]
    print(f"Best solution: {best_solution}")

    # 단일 주문 번들을 최적화
    optimized_solution = optimize_single_order_bundles(best_solution, all_orders, all_riders, dist_mat, K)

    # 최종 solution 출력 형식으로 변환
    final_solution = [
        [bundle.rider.type, bundle.shop_seq, bundle.dlv_seq]
        for bundle in optimized_solution
    ]

    return final_solution

In [ ]:
best_solution = single_run_algorithm(K, all_orders, all_riders, dist_mat, timelimit=60)

Initial best obj = 4189.395


In [ ]:
best_solution

[Bundle(all_orders, BIKE, [13, 51, 44, 36], [51, 13, 44, 36], 99, True),
 Bundle(all_orders, BIKE, [74, 93, 85, 98], [74, 93, 98, 85], 95, True),
 Bundle(all_orders, BIKE, [5, 4, 9], [4, 9, 5], 45, True),
 Bundle(all_orders, BIKE, [28, 32, 35, 53], [28, 35, 53, 32], 77, True),
 Bundle(all_orders, BIKE, [47, 29, 54], [29, 54, 47], 87, True),
 Bundle(all_orders, BIKE, [38, 46, 58, 62], [46, 38, 62, 58], 57, True),
 Bundle(all_orders, BIKE, [91, 90, 84], [90, 84, 91], 93, True),
 Bundle(all_orders, BIKE, [25, 50, 18], [18, 25, 50], 77, True),
 Bundle(all_orders, BIKE, [15, 23, 34, 20], [23, 15, 20, 34], 90, True),
 Bundle(all_orders, BIKE, [40, 45, 67, 82], [45, 40, 67, 82], 71, True),
 Bundle(all_orders, BIKE, [60, 65, 70], [70, 60, 65], 63, True),
 Bundle(all_orders, BIKE, [24, 30, 63], [24, 30, 63], 57, True),
 Bundle(all_orders, BIKE, [1, 3], [3, 1], 56, True),
 Bundle(all_orders, BIKE, [64, 59, 76, 89], [59, 89, 64, 76], 81, True),
 Bundle(all_orders, BIKE, [31, 77, 86], [31, 86, 77]

In [ ]:
def optimize_single_order_bundles(best_solution, all_orders, all_riders, dist_mat, K, max_nearest_bundles=5):
    single_order_bundles = [bundle for bundle in best_solution if len(bundle.shop_seq) == 1]
    remaining_solution = best_solution[:]  # best_solution을 복사하여 사용

    for single_bundle in single_order_bundles:
        single_order_id = single_bundle.shop_seq[0]
        single_order = all_orders[single_order_id]

        # 해당 주문에 가장 가까운 5개의 주문 찾기
        nearest_orders = find_nearest_orders([single_order], all_orders, dist_mat, K, max_nearest_bundles)
        nearest_order_ids = [order.id for order in nearest_orders]

        # 이 주문들이 들어가 있는 번들들 찾기
        associated_bundles = set()  # 중복을 피하기 위해 set 사용
        for order_id in nearest_order_ids:
            for bundle in remaining_solution:  # 남아 있는 솔루션에서 번들 찾기
                if order_id in bundle.shop_seq:
                    associated_bundles.add(bundle)
                    break

        # nearest_order_ids 중 번들에 포함되지 않은 주문이 있는지 확인
        missing_orders = set(nearest_order_ids) - {order_id for bundle in associated_bundles for order_id in bundle.shop_seq}

        # 모든 주문이 포함된 번들을 찾지 못한 경우 추가로 검색
        for order_id in missing_orders:
            for bundle in best_solution:
                if order_id in bundle.shop_seq:
                    associated_bundles.add(bundle)
                    break

        # 현재 번들과 찾은 번들을 합침 (단, single_bundle이 이미 associated_bundles에 있을 경우 제외)
        candidate_bundles = list(associated_bundles)  # set을 list로 변환
        if single_bundle not in associated_bundles:
            candidate_bundles.append(single_bundle)

        candidate_order_ids = [order_id for bundle in candidate_bundles for order_id in bundle.shop_seq]

        # best_solution에서 candidate_bundles에 있는 번들을 제거
        remaining_solution = [bundle for bundle in remaining_solution if bundle not in candidate_bundles]

        # 라이더 선택 및 할당
        while candidate_order_ids:  # 묶이지 않은 주문이 있을 때까지 반복
            rider = assign_riders_with_weighted_probability(all_riders, calculate_efficiencies(K, all_riders, all_orders, dist_mat))
            if rider.available_number > 0:
                new_rider = rider
            else:
                continue

            # 번들로 다시 묶기
            new_bundles, remaining_orders = assign_orders_to_rider(new_rider, [all_orders[i] for i in candidate_order_ids], dist_mat, K, all_orders)
            print(f"new_bundles : {new_bundles}")
            print(f"remaining_order : {remaining_orders}")

            # 기존 번들들과 새로운 번들들 간의 비용 비교
            existing_total_cost = sum(bundle.cost for bundle in candidate_bundles)
            new_total_cost = sum(bundle.cost for bundle in new_bundles)

            if new_total_cost < existing_total_cost:
                remaining_solution.extend(new_bundles)
                candidate_order_ids = [order.id for order in remaining_orders]  # 남아 있는 주문 갱신
            else:
                remaining_solution.extend(candidate_bundles)
                break  # 새로운 번들로 묶지 않는 것이 더 나은 경우 반복 종료

    # 최종 솔루션으로 업데이트
    final_solution = remaining_solution

    # 최종 솔루션의 목적함수 계산 (총 비용)
    final_solution_cost = sum(bundle.cost for bundle in final_solution)

    return final_solution

In [ ]:
final_soluition = optimize_single_order_bundles(best_solution, all_orders, all_riders, dist_mat, K, max_nearest_bundles=5)

final_soluition

KeyboardInterrupt: 

In [ ]:
import numpy as np
import time
import itertools
import random
from itertools import permutations
from util import Bundle, Order, Rider, select_two_bundles, try_merging_bundles, get_total_distance, get_total_volume, test_route_feasibility, get_cheaper_available_riders, try_bundle_rider_changing
import concurrent.futures
import json

problem_file = 'STAGE1_1.json'
# problem_file = "TEST_K50_1.json"
timelimit = 60

# np.random.seed(1)

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

all_orders = [Order(order_info) for order_info in prob['ORDERS']]
all_riders = [Rider(rider_info) for rider_info in prob['RIDERS']]

dist_mat = np.array(prob['DIST'])
for r in all_riders:
    r.T = np.round(dist_mat/r.speed + r.service_time)

In [ ]:
best_solution =  [['WALK', [240], [240]], ['BIKE', [178, 115, 235], [235, 178, 115]], ['BIKE', [143, 223, 150], [150, 143, 223]], ['BIKE', [78, 106, 189], [106, 78, 189]], ['BIKE', [20, 52], [52, 20]], ['BIKE', [117, 48], [117, 48]], ['BIKE', [25, 38, 19], [38, 19, 25]], ['BIKE', [285, 269, 295], [295, 285, 269]], ['BIKE', [103, 67, 93], [103, 67, 93]], ['BIKE', [217, 196, 220], [217, 196, 220]], ['BIKE', [0, 15, 96, 112], [0, 15, 96, 112]], ['BIKE', [101, 87, 166], [87, 101, 166]], ['BIKE', [26, 13, 116], [26, 116, 13]], ['BIKE', [145, 107, 154, 144], [145, 107, 144, 154]], ['BIKE', [37, 1, 92], [1, 92, 37]], ['BIKE', [153, 124, 225, 109], [124, 153, 225, 109]], ['BIKE', [80, 32, 45], [32, 45, 80]], ['BIKE', [248, 203, 195], [203, 248, 195]], ['BIKE', [10, 95, 28, 165], [10, 28, 95, 165]], ['BIKE', [192, 175, 201], [175, 192, 201]], ['BIKE', [65, 241, 77], [77, 65, 241]], ['BIKE', [31, 40, 90, 62], [31, 90, 40, 62]], ['BIKE', [268, 187, 202], [202, 187, 268]], ['BIKE', [147, 242, 171, 208], [242, 147, 171, 208]], ['BIKE', [14, 125, 105], [14, 125, 105]], ['BIKE', [79, 129, 102], [129, 102, 79]], ['BIKE', [41, 91], [91, 41]], ['BIKE', [36, 4, 22], [22, 4, 36]], ['BIKE', [229, 138, 231], [138, 229, 231]], ['BIKE', [281, 234, 287], [287, 234, 281]], ['BIKE', [114, 66, 141, 183], [66, 114, 183, 141]], ['BIKE', [108, 27, 122, 69], [27, 108, 69, 122]], ['BIKE', [163, 256, 257], [257, 163, 256]], ['BIKE', [71, 252], [71, 252]], ['BIKE', [44, 35, 72], [72, 44, 35]], ['BIKE', [7, 9, 81], [9, 81, 7]], ['BIKE', [53, 46], [46, 53]], ['BIKE', [194, 247], [194, 247]], ['BIKE', [155, 254, 233], [155, 233, 254]], ['BIKE', [17, 82, 172], [17, 82, 172]], ['BIKE', [244, 261], [244, 261]], ['BIKE', [132, 211, 238, 227], [211, 132, 238, 227]], ['BIKE', [130, 298, 293], [130, 298, 293]], ['BIKE', [120, 131, 250], [131, 250, 120]], ['BIKE', [219, 249, 279], [219, 249, 279]], ['BIKE', [146, 174, 221], [221, 174, 146]], ['BIKE', [16, 85, 126], [16, 126, 85]], ['BIKE', [64, 43, 12], [43, 12, 64]], ['BIKE', [47, 56, 98], [47, 56, 98]], ['BIKE', [224, 176, 239, 282], [176, 239, 282, 224]], ['BIKE', [149, 168, 270], [149, 270, 168]], ['BIKE', [297, 209, 286], [209, 297, 286]], ['BIKE', [186, 113], [113, 186]], ['BIKE', [253, 214, 272], [214, 253, 272]], ['BIKE', [230, 162, 267], [230, 267, 162]], ['BIKE', [185, 206, 123], [185, 206, 123]], ['BIKE', [255, 288], [255, 288]], ['BIKE', [70, 11, 74], [74, 11, 70]], ['BIKE', [83, 205, 218], [83, 218, 205]], ['BIKE', [21, 139], [21, 139]], ['BIKE', [156, 283], [156, 283]], ['WALK', [271], [271]], ['WALK', [57], [57]], ['CAR', [179, 181, 177], [179, 177, 181]], ['CAR', [23, 5], [23, 5]], ['CAR', [42, 118, 39], [42, 39, 118]], ['CAR', [159, 137], [137, 159]], ['CAR', [169, 263], [169, 263]], ['CAR', [289, 243], [243, 289]], ['CAR', [119, 173], [173, 119]], ['CAR', [226, 84], [84, 226]], ['CAR', [6, 50], [6, 50]], ['CAR', [29, 160], [29, 160]], ['CAR', [199, 136, 128], [136, 199, 128]], ['CAR', [18, 55], [18, 55]], ['CAR', [180, 228, 200], [180, 200, 228]], ['CAR', [182, 190], [182, 190]], ['CAR', [140, 284, 273], [140, 284, 273]], ['CAR', [73, 100], [73, 100]], ['CAR', [61], [61]], ['CAR', [86, 110], [110, 86]], ['CAR', [212, 292], [292, 212]], ['CAR', [97], [97]], ['CAR', [276, 258], [276, 258]], ['CAR', [161, 135, 198], [135, 161, 198]], ['CAR', [75], [75]], ['CAR', [30], [30]], ['CAR', [148, 278, 215], [148, 278, 215]], ['CAR', [51], [51]], ['CAR', [275, 265], [275, 265]], ['CAR', [204], [204]], ['CAR', [59, 33], [33, 59]], ['CAR', [294, 266], [266, 294]], ['CAR', [24, 89], [89, 24]], ['CAR', [170, 157, 158], [158, 170, 157]], ['CAR', [104, 184, 237], [184, 104, 237]], ['CAR', [8], [8]], ['CAR', [99], [99]], ['CAR', [259, 290], [290, 259]], ['CAR', [280, 299], [299, 280]], ['CAR', [236, 207], [207, 236]], ['CAR', [58], [58]], ['CAR', [94, 167], [167, 94]], ['CAR', [232, 251], [232, 251]], ['CAR', [142, 191, 213], [142, 213, 191]], ['CAR', [151, 222], [151, 222]], ['CAR', [34, 76], [34, 76]], ['CAR', [193, 264], [264, 193]], ['CAR', [121, 134], [121, 134]], ['CAR', [260], [260]], ['CAR', [188], [188]], ['CAR', [164, 210, 245], [164, 210, 245]], ['CAR', [133], [133]], ['CAR', [60, 63], [63, 60]], ['CAR', [3, 54], [3, 54]], ['CAR', [216, 152], [216, 152]], ['CAR', [246, 291], [246, 291]], ['CAR', [274, 277], [277, 274]], ['CAR', [197], [197]], ['CAR', [296], [296]], ['CAR', [49], [49]], ['CAR', [2, 111], [2, 111]], ['CAR', [68], [68]], ['CAR', [262], [262]], ['CAR', [88], [88]], ['CAR', [127], [127]]]
# 카운트 딕셔너리 초기화
counts = {
    "BIKE": 0,
    "WALK": 0,
    "CAR": 0
}

# 각 번들 타입을 카운트
for bundle in best_solution:
    rider_type = bundle[0]
    if rider_type == "BIKE":
        counts["BIKE"] += 1
    elif rider_type == "WALK":
        counts["WALK"] += 1
    elif rider_type == "CAR":
        counts["CAR"] += 1

# 결과 출력
print("Bike Riders used:", counts["BIKE"])
print("Walk Riders used:", counts["WALK"])
print("Car Riders used:", counts["CAR"])

Bike Riders used: 60
Walk Riders used: 3
Car Riders used: 63
